In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import csv
import numpy as np
import ephem
from datetime import datetime

%matplotlib inline
from matplotlib import pyplot as plt
import scipy.stats as stats

social_media_path = "OSF_socialmedia_data.csv"
social_media = pd.read_csv(social_media_path)
social_media.rename(columns = {"Unnamed: 0": "Row"}, inplace = True)
social_media.set_index("Row", inplace=True)
social_media.head()

,Participant,Date,Day,Time,Session.Name,Notification.No,LifePak.Download.No,Responded,Completed.Session,Session.Instance,...,Loneliness,Concentrat,LossOfInt,Inferior,Hopeless,Stress,PSMU,AutoPSMU,News,Active
Row,,,,,,,,,,,,,,,,,,,,,
1,115091,5/2/2017,Tuesday,10:00:06,Daily questions,1,1,1,1,1.0,...,61.0,64.0,68.0,50.0,56.0,73.0,66.0,66,39.0,38.0
2,115091,5/2/2017,Tuesday,12:02:48,Daily questions,2,1,1,1,2.0,...,57.0,70.0,58.0,37.0,56.0,67.0,0.0,#skipped#,27.0,23.0
3,115091,5/2/2017,Tuesday,14:03:02,Daily questions,3,1,1,1,3.0,...,44.0,70.0,67.0,26.0,38.0,39.0,63.0,56,34.0,36.0
4,115091,5/2/2017,Tuesday,16:34:28,Daily questions,4,1,1,1,4.0,...,56.0,79.0,89.0,30.0,20.0,60.0,67.0,68,29.0,23.0
5,115091,5/2/2017,Tuesday,18:00:00,Daily questions,5,1,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
social_media.dtypes

Participant                          int64
Date                                object
Day                                 object
Time                                object
Session.Name                        object
Notification.No                      int64
LifePak.Download.No                  int64
Responded                            int64
Completed.Session                    int64
Session.Instance                   float64
Session.Instance.Response.Lapse     object
Reminders.Delivered                  int64
Instr_DQs                          float64
Fatigue                            float64
DeprMood                           float64
Loneliness                         float64
Concentrat                         float64
LossOfInt                          float64
Inferior                           float64
Hopeless                           float64
Stress                             float64
PSMU                               float64
AutoPSMU                            object
News       

In [4]:
def get_phase_on_day(dt):
  """Returns a floating-point number from 0-1. where 0=new, 0.5=full, 1=new"""
  #Ephem stores its date numbers as floating points, which the following uses
  #to conveniently extract the percent time between one new moon and the next
  #This corresponds (somewhat roughly) to the phase of the moon.

  #Use Year, Month, Day as arguments
  date=ephem.Date(datetime.date(dt))

  nnm = ephem.next_new_moon    (date)
  pnm = ephem.previous_new_moon(date)

  lunation=(date-pnm)/(nnm-pnm)

  #Note that there is a ephem.Moon().phase() command, but this returns the
  #percentage of the moon which is illuminated. This is not really what we want.

  return lunation

In [5]:
pd.to_datetime(social_media["Date"]).map(get_phase_on_day)

Row
1        0.187258
2        0.187258
3        0.187258
4        0.187258
5        0.187258
           ...   
12241    0.948577
12242    0.948577
12243    0.948577
12244    0.948577
12245    0.948577
Name: Date, Length: 12245, dtype: float64

In [6]:
social_media["LunarPhase"] = pd.to_datetime(social_media["Date"]).map(get_phase_on_day)
social_media.head()

,Participant,Date,Day,Time,Session.Name,Notification.No,LifePak.Download.No,Responded,Completed.Session,Session.Instance,...,Concentrat,LossOfInt,Inferior,Hopeless,Stress,PSMU,AutoPSMU,News,Active,LunarPhase
Row,,,,,,,,,,,,,,,,,,,,,
1,115091,5/2/2017,Tuesday,10:00:06,Daily questions,1,1,1,1,1.0,...,64.0,68.0,50.0,56.0,73.0,66.0,66,39.0,38.0,0.187258
2,115091,5/2/2017,Tuesday,12:02:48,Daily questions,2,1,1,1,2.0,...,70.0,58.0,37.0,56.0,67.0,0.0,#skipped#,27.0,23.0,0.187258
3,115091,5/2/2017,Tuesday,14:03:02,Daily questions,3,1,1,1,3.0,...,70.0,67.0,26.0,38.0,39.0,63.0,56,34.0,36.0,0.187258
4,115091,5/2/2017,Tuesday,16:34:28,Daily questions,4,1,1,1,4.0,...,79.0,89.0,30.0,20.0,60.0,67.0,68,29.0,23.0,0.187258
5,115091,5/2/2017,Tuesday,18:00:00,Daily questions,5,1,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.187258


In [7]:

bins = [0, 0.0339, 0.0678, 0.1017, 0.1356, 0.1695, 0.2034, 0.2373, 0.2712, 0.3051, 0.3390, 0.3729, 0.4068, 0.4407, 0.4746, 0.5085, 0.5424, 0.5763, 0.6102, 0.6441, 0.6780, 0.7119, 0.7458, 0.7797, 0.8136, 0.8475, 0.8814, 0.9153, 0.9492, 0.9831, 1.0000]

LunarPhaseDayGroups = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"]

social_media["LunarPhaseDay"] = pd.cut(social_media["LunarPhase"], bins, labels=LunarPhaseDayGroups, include_lowest=True)
social_media.head(100)

,Participant,Date,Day,Time,Session.Name,Notification.No,LifePak.Download.No,Responded,Completed.Session,Session.Instance,...,LossOfInt,Inferior,Hopeless,Stress,PSMU,AutoPSMU,News,Active,LunarPhase,LunarPhaseDay
Row,,,,,,,,,,,,,,,,,,,,,
1,115091,5/2/2017,Tuesday,10:00:06,Daily questions,1,1,1,1,1.0,...,68.0,50.0,56.0,73.0,66.0,66,39.0,38.0,0.187258,6
2,115091,5/2/2017,Tuesday,12:02:48,Daily questions,2,1,1,1,2.0,...,58.0,37.0,56.0,67.0,0.0,#skipped#,27.0,23.0,0.187258,6
3,115091,5/2/2017,Tuesday,14:03:02,Daily questions,3,1,1,1,3.0,...,67.0,26.0,38.0,39.0,63.0,56,34.0,36.0,0.187258,6
4,115091,5/2/2017,Tuesday,16:34:28,Daily questions,4,1,1,1,4.0,...,89.0,30.0,20.0,60.0,67.0,68,29.0,23.0,0.187258,6
5,115091,5/2/2017,Tuesday,18:00:00,Daily questions,5,1,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.187258,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,115091,5/15/2017,Monday,18:00:00,Daily questions,5,1,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.630773,19
97,115091,5/15/2017,Monday,20:10:42,Daily questions,6,1,1,1,77.0,...,20.0,16.0,8.0,3.0,8.0,#skipped#,13.0,13.0,0.630773,19
98,115091,5/15/2017,Monday,22:27:28,Daily questions,7,1,1,1,78.0,...,72.0,9.0,12.0,67.0,24.0,11,11.0,10.0,0.630773,19


In [8]:
social_media.dtypes

Participant                           int64
Date                                 object
Day                                  object
Time                                 object
Session.Name                         object
Notification.No                       int64
LifePak.Download.No                   int64
Responded                             int64
Completed.Session                     int64
Session.Instance                    float64
Session.Instance.Response.Lapse      object
Reminders.Delivered                   int64
Instr_DQs                           float64
Fatigue                             float64
DeprMood                            float64
Loneliness                          float64
Concentrat                          float64
LossOfInt                           float64
Inferior                            float64
Hopeless                            float64
Stress                              float64
PSMU                                float64
AutoPSMU                        